# Step 1 : Convert Keras model into TensorRT model
**For more detail, checkout [How to run Keras model on Jetson Nano](https://www.dlology.com/blog/how-to-run-keras-model-on-jetson-nano/) | DLology Blog**

Changing the runtime to Python2
to get use of Tensorflow version to 1.14.0

In [0]:
!pip install tensorflow-gpu==1.14.0
import tensorflow as tf
print(tf.__version__)

In [0]:
import tensorflow.contrib.tensorrt as trt

In [0]:
# Download an image for prediction.

!wget --quiet https://raw.githubusercontent.com/Tony607/tf_jetson_nano/master/data/elephant.jpg

In [0]:

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as Net
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np
import os

# Optional image to test model prediction.
img_path = '/content/elephant.jpg'
model_path = '/content/model'

#os.makedirs(model_path, exist_ok=True)
# Path to save the model h5 file.
model_fname = os.path.join(model_path, 'model.h5')

#os.makedirs(model_path, exist_ok=True)

img_height = 224

model = Net(weights='imagenet')


# Load the image for prediction.
img = image.load_img(img_path, target_size=(img_height, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

# Save the h5 file to path specified.
model.save(model_fname)

### Benchmark Keras prediction speed.

In [0]:
import time
times = []
for i in range(20):
    start_time = time.time()
    preds = model.predict(x)
    delta = (time.time() - start_time)
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1/mean_delta
print('average(sec):{},fps:{}'.format(mean_delta,fps))

# Clear any previous session.
tf.keras.backend.clear_session()

## Freeze graph, generate `.pb` file.
Take a notes of the input and output nodes names printed in the output, we will need them when converting `TensorRT` graph and prediction.

For Keras MobileNetV2, they are,
```
['input_1'] ['Logits/Softmax']
```


In [0]:
# force reset ipython namespaces
%reset -f

import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model


# Clear any previous session.
tf.keras.backend.clear_session()

save_pb_dir = './model'
model_fname = './model/model.h5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

## Optimize with TensorRT
Save the result as `.pb` file.

*Notes: optimizing TensorRT graph can also be executed on Jetson Nano, but it is very slow.*

In [0]:
import tensorflow.contrib.tensorrt as trt

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

In [0]:
graph_io.write_graph(trt_graph, "/content/model/",
                     "trt_graph.pb", as_text=False)

In [0]:
!ls model -alh

### Download the tensorRT graph `.pb` file from colab to your local machine.

In [0]:
from google.colab import files

files.download('./model/trt_graph.pb')

**Next step**: transfer the `trt_graph.pb` to your Jetson Nano, load it up and make predictions.


`Step2_keras-jetson-ImageNet-predict.ipynb`